In [1]:
import numpy as np
import pandas as pd

In [2]:
"""
Import the dataset
"""
df = pd.read_csv('/content/diabetes.csv')

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
"""
Check NaN value appeared or not
"""
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [4]:
"""
Split the dataset into train and test
"""
from sklearn.model_selection import train_test_split

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, y,\
                                                    test_size = 0.20,\
                                                    random_state = 42,\
                                                    shuffle = True)

print('X_train shape is # {}'.format(X_train.shape),'\n')
print('X_test  shape is # {}'.format(X_test.shape),'\n')
print('y_train shape is # {}'.format(y_train.shape),'\n')
print('y_test  shape is # {}'.format(y_test.shape))

X_train shape is # (614, 8) 

X_test  shape is # (154, 8) 

y_train shape is # (614,) 

y_test  shape is # (154,)


In [5]:
X_train[0:2]

array([[  2.   ,  84.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.304,
         21.   ],
       [  9.   , 112.   ,  82.   ,  24.   ,   0.   ,  28.2  ,   1.282,
         50.   ]])

In [6]:
"""
Import the DataLoader function so that we can initialize the batch_size
"""
import torch
from torch.utils.data import Dataset, DataLoader

data_loader = DataLoader(dataset = list(zip(X_train, y_train)),\
                         batch_size = 16,\
                         shuffle = True)
data, labels = next(iter(data_loader))

print("The shape of singular batch size of training # {}".format(data.size()))
print("The shape of singular batch size of testing  # {}".format(labels.size()))

The shape of singular batch size of training # torch.Size([16, 8])
The shape of singular batch size of testing  # torch.Size([16])


#Creating the ANN architecture

In [7]:
"""
Import the necessity liabries.
"""
import torch.nn as nn

"""
Creating a class that will inherit the parent class named `nn.Module`.
"""
class DiabeticsClassifier(nn.Module):
  """
  Create a constructors that might take some of the parameters. Here, I did not pass anything.
  """
  def __init__(self):
    """
    Call the super() functions that has some properties
    """
    super().__init__()
    """
    Create the first hidden layer with its activation functions
    """
    self.hidden_1 = nn.Linear(in_features = 8,\
                              out_features = 128,\
                              bias = True)
    self.activation_1 = nn.ReLU()

    """
    Create the second hidden hidden layer with its activation function
    """
    self.hidden_2 = nn.Linear(in_features = 128,\
                              out_features = 64,\
                              bias = True)
    self.activation_2 = nn.ReLU()

    """
    Create the output layer with its activation function
    """
    self.output = nn.Linear(in_features = 64,\
                            out_features = 1,\
                            bias = True)
    self.activation_output = nn.Sigmoid()

  def forward(self, x):
    """
    Formula of forward propagation `activation(W^T + B)`; `x` means training dataset
    """
    x = self.activation_1(self.hidden_1(x))
    x = self.activation_2(self.hidden_2(x))
    x = self.activation_output(self.output(x))

    return x

"""
Create an object of this class
"""

model = DiabeticsClassifier()

print(model)

DiabeticsClassifier(
  (hidden_1): Linear(in_features=8, out_features=128, bias=True)
  (activation_1): ReLU()
  (hidden_2): Linear(in_features=128, out_features=64, bias=True)
  (activation_2): ReLU()
  (output): Linear(in_features=64, out_features=1, bias=True)
  (activation_output): Sigmoid()
)


In [8]:
print(model.parameters)

<bound method Module.parameters of DiabeticsClassifier(
  (hidden_1): Linear(in_features=8, out_features=128, bias=True)
  (activation_1): ReLU()
  (hidden_2): Linear(in_features=128, out_features=64, bias=True)
  (activation_2): ReLU()
  (output): Linear(in_features=64, out_features=1, bias=True)
  (activation_output): Sigmoid()
)>


In [9]:
print(model.parameters())

<generator object Module.parameters at 0x7a883ad962d0>


In [10]:
from torchsummary import summary

In [11]:
"""
Create the model summary. This will take two parameters(1. Model 2. shape of training/input data)
"""
summary(model, X_train.shape)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1             [-1, 614, 128]           1,152
              ReLU-2             [-1, 614, 128]               0
            Linear-3              [-1, 614, 64]           8,256
              ReLU-4              [-1, 614, 64]               0
            Linear-5               [-1, 614, 1]              65
           Sigmoid-6               [-1, 614, 1]               0
Total params: 9,473
Trainable params: 9,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.02
Forward/backward pass size (MB): 1.81
Params size (MB): 0.04
Estimated Total Size (MB): 1.86
----------------------------------------------------------------
